In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.support.ui import Select

from fuzzywuzzy import fuzz
from time import sleep
import operator


C:\Users\arvind.baranwal\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:

def get_browser():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('log-level=3')
    browser = webdriver.Chrome(chrome_options=chrome_options)
    return browser


In [4]:

def clean(name):
    name = name.lower()
    junk_words = [" corporation"," inc"," ltd"," pvt","."," llc"," llp","&amp",";",","]
    for word in junk_words:
        name = name.replace(word,"")
    name = name.strip()
    return name


In [13]:

import pandas as pd
data = pd.read_excel('C:/Users/arvind.baranwal/Downloads/12th july/JanuaryRenewals pulled 7-10-19_CleanedV.01.xlsx',sheetname = 'Cleaned January Renewals')
data.shape



(423, 19)

In [14]:
data.head()

,Sr. No.,Identifier,Given Account_Name,Given Street,Given City,Given State,Given Zip,Given Country,LOB,Renewal Date,Input Type,PO BOX,Unique/Duplicate,Cleaned Account_Name,Cleaned Street,Cleaned City,Cleaned State,Cleaned Zip,Cleaned Country
0,830,JAN20REN830,UTILITY CONTRACTORS INC,PO BOX 9592,WICHITA,KS,67277,USA,Auto Lines,2020-01-01,Company,Y,Unique,UTILITY CONTRACTORS INC,PO BOX 9592,WICHITA,KS,67277,USA
1,829,JAN20REN829,INDUSTRIAL SERVICES COMPANY INC,PO BOX 9286,SPRINGFIELD,MO,65803,USA,Auto Lines,2020-01-01,Company,Y,Unique,INDUSTRIAL SERVICES COMPANY INC,PO BOX 9286,SPRINGFIELD,MO,65803,USA
2,973,JAN20REN973,"OZARK STRIPING COMPANY, INC.",PO BOX 847,OZARK,AL,36361,USA,WC Lines,2020-01-01,Company,Y,Unique,"OZARK STRIPING COMPANY, INC.",PO BOX 847,OZARK,AL,36361,USA
3,155,JAN20REN155,SOUTHCREEK MANAGEMENT COMPANY,PO BOX 8250,LEXINGTON,KY,40533,USA,Auto Lines,2020-01-01,Company,Y,Unique,SOUTHCREEK MANAGEMENT COMPANY,PO BOX 8250,LEXINGTON,KY,40533,USA
4,142,JAN20REN142,TASLER INC,PO BOX 726,WEBSTER CITY,IA,50595,USA,Auto Lines,2020-01-01,Company,Y,Unique,TASLER INC,PO BOX 726,WEBSTER CITY,IA,50595,USA


In [15]:

original = data['Cleaned Account_Name'].tolist()
query_company = data['Cleaned Account_Name'].tolist()
query_country = data['Cleaned Country'].tolist()


driver = get_browser()


C:\Users\arvind.baranwal\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


In [16]:
driver = get_browser()
url = "https://www.kununu.com/us/search#/"

# query_company = ['basement system inc.']
# query_state =['','Dallas','']



company_name =[]
review_link =[]
score_max=[]
original_name=[]

# for i in range(13,len(query_company)):
for i in range(0,len(query_company)):
    
#     driver.implicitly_wait(3) 
    driver.get(url)

    driver.find_element_by_xpath("""//*[@id="home_profile_value"]""").clear()
    co_name = driver.find_element_by_xpath("""//*[@id="home_profile_value"]""")
    co_name.send_keys(query_company[i].lower())
    
#     driver.find_element_by_xpath("""//*[@id="home_location_value"]""").clear()
#     city = driver.find_element_by_xpath("""//*[@id="home_location_value"]""")
#     city.send_keys(query_city[i].lower())
    
    select = Select(driver.find_element_by_xpath("""//*[@id="countrySelect"]"""))
    select.select_by_visible_text('United States')
    
    sleep(3)

    soup = BeautifulSoup(driver.page_source, "html.parser")

    try:

        ### searching for names on the first page only.
        name=[]
        for n in soup.find_all("h2",{"class":"company-title-with-label"}):
            name.append(n.text.strip())
        
        print(name)
#         list_company.append(name)
        
        # do we need location -- to match?
#         location=[]
#         for l in soup.find_all("p",{"class":"ng-binding"}): 
#             location.append(l.text.strip())

#         location = location[0::2]
#         print(location)

        ## storing list of companies fuzzy score with the query company name
        score=[]
        for s in name:
            print(s)
            score.append(fuzz.ratio(clean(query_company[i]).lower(),clean(s).lower()))
        print(score)
        
#         list_score.append(score)
        
        ## cut off as 90-- if more than one cutoff is 90-- choosing max value. What if max value appears more than one time??
        values={}
        for k,v in enumerate(score):
            if v>79:
                values[k] = v

        position = max(values.items(), key=operator.itemgetter(1))[0]
        score_maxvalue = max(values.items(), key=operator.itemgetter(1))[1]
        
        
        ## appending with 
        NAME = name[position]
        ORI_NAME = original[i]
        LINK = "https://www.kununu.com"+soup.find_all("a",{"class":"link-unstyled ng-binding"},href=True)[position]['href']
        SCORE= score_maxvalue
        
    except:
        NAME=""
        ORI_NAME = original[i]
        LINK=""
        SCORE=""
    company_name.append(NAME)
    review_link.append(LINK)
    score_max.append(SCORE)
    original_name.append(ORI_NAME)
    
    driver.refresh

driver.quit()

pd.DataFrame({'company_name':company_name,'original_name':original_name,'review_link':review_link,'max_score':score_max}).to_excel('C:/Users/arvind.baranwal/Downloads/12th july/Jan_Renewal_Prospects_kununu_top.xlsx')


C:\Users\arvind.baranwal\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


['Utility Data Contractors, Inc.']
Utility Data Contractors, Inc.
[88]
['Clean Harbors', 'H&M', 'thyssenkrupp', 'Aramark', 'General Electric', 'Caterpillar', 'Cummins', 'Toyota', 'KBR', 'Marshalls']
Clean Harbors
H&M
thyssenkrupp
Aramark
General Electric
Caterpillar
Cummins
Toyota
KBR
Marshalls
[20, 7, 26, 18, 23, 26, 18, 12, 7, 22]
['Ozark Striping Company, Inc.']
Ozark Striping Company, Inc.
[100]
[]
[]
[]
[]
[]
[]
['Toyota']
Toyota
[18]
[]
[]
[]
[]
['Kleenco Maintenance Construction Inc']
Kleenco Maintenance Construction Inc
[97]
[]
[]
[]
[]
[]
[]
['Allen Tate Co., Inc.']
Allen Tate Co., Inc.
[100]
[]
[]
[]
[]
[]
[]
[]
[]
['Siemens AG', 'Deutsche Bahn', 'Deutsche Telekom AG', 'Netto Marken-Discount AG & Co. KG', 'Daimler AG', 'Bosch Gruppe', 'Amadeus FiRe AG', 'Lidl', 'FERCHAU GmbH', 'Deutsche Post DHL Group', 'UPS', 'Vodafone GmbH', 'TEMPTON', 'Continental AG', 'SAP SE', 'Allianz Deutschland AG', 'Randstad Deutschland GmbH & Co KG', 'Commerzbank AG']
Siemens AG
Deutsche Bahn
Deutsc

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['Flowserve', 'Ceiling Professionals International Inc.', 'Alp Lighting & Ceiling Products, Inc.']
Flowserve
Ceiling Professionals International Inc.
Alp Lighting & Ceiling Products, Inc.
[12, 37, 41]
['IMEG Corp']
IMEG Corp
[100]
["McDonald's", 'Hatch']
McDonald's
Hatch
[8, 50]
['Bpi Outdoor Living, Inc', 'Bpl Plasma, Inc.', 'Mpi Research, Inc.', 'LPI Inc.', 'Bti Solutions, Inc.', 'FPI MANAGEMENT', 'FPI Management, Inc.', 'CPI Security Systems, Inc.', 'Blood Systems Inc. (BSI)', 'BHI Inc', 'BMI', 'The Lavoi Corporation/Epi Breads, Inc', 'Dpi, Inc.', 'Bci, Inc.', 'BI, Inc', 'Amazon', 'Perkins Restaurant and Bakery', 'API Outsourcing, Inc.']
Bpi Outdoor Living, Inc
Bpl Plasma, Inc.
Mpi Research, Inc.
LPI Inc.
Bti Solutions, Inc.
FPI MANAGEMENT
FPI Management, Inc.
CPI Security Systems, Inc.
Blood Systems Inc. (BSI)
BHI Inc
BMI
The Lavoi Corporation/Epi Breads, Inc
Dpi, Inc.
Bci, Inc.
BI, Inc
Amazon
Perkins Restaurant and Bakery
API Outsourcing, Inc.
[29, 31

['Allstate', 'Walmart', 'Kemper', 'Hilton']
Allstate
Walmart
Kemper
Hilton
[29, 6, 19, 19]
['Southwest Ltc - Management Services, LLC', 'USDM Life Sciences', 'Essendant Management Services LLC', 'Waste Management', 'Receivables Performance Management, LLC', 'Alliance Inspection Management', 'Fortis Management Group, LLC', 'Progressive', 'Lockheed Martin', 'Accenture', 'Intercontinental Corporation', 'Apartment Management Consultants LLC', 'NRP Group', 'Cobham Advanced Electronic Solutions', 'Frontier Management LLC', 'Pinnacle Property Management, LLC', 'RSM US LLP', 'Ledic Management Group, LLC']
Southwest Ltc - Management Services, LLC
USDM Life Sciences
Essendant Management Services LLC
Waste Management
Receivables Performance Management, LLC
Alliance Inspection Management
Fortis Management Group, LLC
Progressive
Lockheed Martin
Accenture
Intercontinental Corporation
Apartment Management Consultants LLC
NRP Group
Cobham Advanced Electronic Solutions
Frontier Management LLC
Pinnacle 

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['Dynamic Materials Corporation']
Dynamic Materials Corporation
[100]
[]
[]
[]
[]
[]
[]
['UPS', 'Verizon Wireless']
UPS
Verizon Wireless
[10, 41]
[]
[]
[]
[]
['Schulze & Burch Biscuit Co.']
Schulze & Burch Biscuit Co.
[93]
['Special Olympics Texas, Inc.']
Special Olympics Texas, Inc.
[84]
['Khk Group, Ltd.']
Khk Group, Ltd.
[100]
['PepsiCo', 'Coca-Cola Refreshments', 'Office Depot Office Max', 'The Coca-Cola Company', 'Tim Hortons', 'Hard Rock International', 'Randstad USA', 'Walmart', 'FedEx', 'Verizon Wireless', 'HARD ROCK CAFE INTERNATIONAL (USA), INC.', 'YMCA', 'Nissan']
PepsiCo
Coca-Cola Refreshments
Office Depot Office Max
The Coca-Cola Company
Tim Hortons
Hard Rock International
Randstad USA
Walmart
FedEx
Verizon Wireless
HARD ROCK CAFE INTERNATIONAL (USA), INC.
YMCA
Nissan
[13, 33, 19, 34, 32, 32, 60, 13, 0, 33, 38, 33, 29]
[]
[]
[]
[]
[]
[]
['Tropical Nut & Fruit Co.']
Tropical Nut & Fruit Co.
[100]
[]
[]
[]
[]
['Material Installations, Inc.

## built condensed function for getting all matches

In [33]:
driver = get_browser()

def get_browser():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('log-level=3')
    browser = webdriver.Chrome(chrome_options=chrome_options)
    return browser

def clean(name):
    name = name.lower()
    junk_words = [" corporation"," inc"," ltd"," pvt","."," llc"," llp","&amp",";",","]
    for word in junk_words:
        name = name.replace(word,"")
    name = name.strip()
    return name

def candidates_kununu_1(companyname,match_cutoff):
    driver = get_browser()
    url = "https://www.kununu.com/us/search#/"

    driver.get(url)

    driver.find_element_by_xpath("""//*[@id="home_profile_value"]""").clear()
    co_name = driver.find_element_by_xpath("""//*[@id="home_profile_value"]""")
    co_name.send_keys(companyname.lower())
    
#     driver.find_element_by_xpath("""//*[@id="home_location_value"]""").clear()
#     city = driver.find_element_by_xpath("""//*[@id="home_location_value"]""")
#     city.send_keys(query_city[i].lower())
    
    select = Select(driver.find_element_by_xpath("""//*[@id="countrySelect"]"""))
    select.select_by_visible_text('United States')
    
    sleep(3)

    soup = BeautifulSoup(driver.page_source, "html.parser")

    try:

        ### searching for names on the first page only.
        name=[]
        for n in soup.find_all("h2",{"class":"company-title-with-label"}):
            name.append(n.text.strip())

        ## storing list of companies fuzzy score with the query company name
        scores=[]
        found_names = []
        review_links = []
        position = 0
        for s in name:
            match_score = fuzz.ratio(clean(companyname).lower(),clean(s).lower()) 
            if match_score > match_cutoff:
                found_names.append(s)
                review_links.append("https://www.kununu.com"+soup.find_all("a",{"class":"link-unstyled ng-binding"},href=True)[position]['href'])
                scores.append(match_score)
            position+=1             
        #         print('names after matching cut-off')
        #         print(len(found_names))
        #         print(found_names)
        #         print('review links')
        #         print(len(review_links))
        #         print(review_links)
        #         print('matching scores')
        #         print(len(scores))
        #         print(scores)

    except:
        found_names = []
        review_links = []
        scores=[]  
    
    driver.refresh 
    return found_names, review_links, scores

    

C:\Users\arvind.baranwal\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


In [34]:
candidates_kununu_1('BARSCO, INC.',0)

C:\Users\arvind.baranwal\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


(['Barco, Inc.',
  'tarsco',
  'Barnsco, Inc.',
  'Barco Inc (Rancho Cordova, CA / United States)'],
 ['https://www.kununu.com/us/barco-inc/reviews',
  'https://www.kununu.com/us/tarsco1/reviews',
  'https://www.kununu.com/us/barnsco-inc/reviews',
  'https://www.kununu.com/us/barco-inc-rancho-cordova-ca-united-states/reviews'],
 [91, 83, 92, 21])

## built condensed function for getting all matches (without using selenium, by forming the search url)

In [36]:
def candidates_kununu_2(companyname,match_cutoff):
    driver = get_browser()
    url = "https://www.kununu.com/us/search#/?q="+str(companyname)+"&country=COUNTRY_US"

    driver.get(url)
    sleep(3)

    soup = BeautifulSoup(driver.page_source, "html.parser")

    try:

        ### searching for names on the first page only.
        name=[]
        for n in soup.find_all("h2",{"class":"company-title-with-label"}):
            name.append(n.text.strip())
        ## submitting name & location

        ## storing list of companies fuzzy score with the query company name
        scores=[]
        found_names = []
        review_links = []
        position = 0
        for s in name:
            match_score = fuzz.ratio(clean(companyname).lower(),clean(s).lower()) 
            if match_score > match_cutoff:
                found_names.append(s)
                review_links.append("https://www.kununu.com"+soup.find_all("a",{"class":"link-unstyled ng-binding"},href=True)[position]['href'])
                scores.append(match_score)
            position+=1             
        #         print('names after matching cut-off')
        #         print(len(found_names))
        #         print(found_names)
        #         print('review links')
        #         print(len(review_links))
        #         print(review_links)
        #         print('matching scores')
        #         print(len(scores))
        #         print(scores)

    except:
        found_names = []
        review_links = []
        scores=[]  
    
    driver.refresh 
    return found_names, review_links, scores

In [40]:
candidates_kununu_2('BARSCO, INC.',0)

C:\Users\arvind.baranwal\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


(['Barco, Inc.',
  'tarsco',
  'Barnsco, Inc.',
  'Barco Inc (Rancho Cordova, CA / United States)'],
 ['https://www.kununu.com/us/barco-inc/reviews',
  'https://www.kununu.com/us/tarsco1/reviews',
  'https://www.kununu.com/us/barnsco-inc/reviews',
  'https://www.kununu.com/us/barco-inc-rancho-cordova-ca-united-states/reviews'],
 [91, 83, 92, 21])